<a href="https://colab.research.google.com/github/degartHub/nocountry-h12-25-equipo27-datascience/blob/main/Notebook_Producto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRODUCTO FINAL

## IMPORTS

### Librerías Utilizadas

Se presentan todas la librerías utilizadas junto a su función en el cuaderno a continuación.

In [ ]:
from requests.adapters import HTTPAdapter                 # Controlar la reconexión automática, límites de conexión y sesiones persistentes.
from urllib3.util.retry import Retry                      # Configurar reintentos automáticos de solicitudes a la API.
from datetime import datetime, timedelta                  # Operar con diferencias de tiempo.
from sklearn.model_selection import train_test_split      # Dividir conjuntos de datos en subconjuntos de entrenamiento y prueba.
from sklearn.preprocessing import OneHotEncoder           # Convertir variables categóricas en variables numéricas con codificación one-hot.
from sklearn.impute import SimpleImputer                  # Manejar valores faltantes reemplazándolos ya sea con la media, mediana, moda o un valor constante.
from sklearn.calibration import CalibratedClassifierCV    # Calibrar las probabilidades producidas por un modelo de clasificación.
from sklearn.ensemble import GradientBoostingClassifier   # Entrenar modelos de clasificación basados en ensamblado de árboles de decisión mediante boosting.

from sklearn.metrics import (
    accuracy_score,                                       # Medir la proporción de predicciones correctas realizadas por el modelo respecto al total de predicciones.
    precision_score,                                      # Medir la proporción de predicciones positivas correctas sobre el total de predicciones positivas realizadas por el modelo.
    recall_score,                                         # Medir la proporción de casos positivos reales que el modelo logra identificar correctamente.
    f1_score,                                             # Proporcionar una medida equilibrada del rendimiento del modelo.
    roc_auc_score,                                         # Evaluar modelos de clasificación
    classification_report                                 # Genera un informe detallado con las anteriores métricas para cada clase del modelo.
)
import pandas as pd                                       # Leer, limpiar, transformar, analizar y resumir datos.
import numpy as np                                        # Realizar calculos numéricos.

import logging                                            # Depurar, monitorear y documentar el comportamiento en ciertas partes del código.
import requests                                           # Realizar solicitudes HTTP para descargar datos de la API.
import os                                                 # Guardar cache de archivos y no depender de descargas repetidas.
import shutil                                             # Eliminar caché.
import joblib                                             # Guardar y cargar modelos entrenados


## CARGA

In [ ]:
# ================================
# CARGA DE DATOS (DE)
# ================================
# Dataset a limpiar para realizar el modelo predictivo

import pandas as pd

url="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Airlines.csv"
df = pd.read_csv(url)

### Descripción del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


Columnas presentes en el dataset:

- <u>**id**</u>= Identifica la fila del registro.

- <u>**Airline**</u>= Aerolínea.

- <u>**Flight**</u>= Número de la aeronave.

- <u>**Airport From**</u>= Aeropuerto de origen.

- <u>**Airport To**</u>= Aeropuerto de destino.

- <u>**DayOfWeek**</u>= Día de la semana (en números).

- <u>**Time**</u>= Hora de salida medida en minutos a partir de la medianoche (rango de [10,1439], lo que podría ser el equivalente a un día).

- <u>**Lenght**</u>= Duración del vuelo en minutos.

- <u>**Delay**</u>= Con retraso (1), sin retraso (0).

### Selección y Limpieza de datos

In [ ]:
#Se toman datos aleatorios de la tabla para entender su contenido.

df.sample(n=5)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
377772,377773,EV,5072,BMI,ATL,3,966,104,1
334371,334372,OO,6432,SLC,DEN,1,615,85,0
350295,350296,WN,554,RSW,MDW,2,495,180,0
13432,13433,B6,1417,AUS,LGB,3,1050,195,0
344454,344455,WN,617,CMH,MDW,1,1140,75,1


#### Columnas eliminadas

- ID: Es un identificador para la tabla en sí
- Flight: Identifica el número de avión, no es relevante.

In [ ]:
df = df.drop(columns=["id", "Flight"])

#### Muestra de datos

Al tener un total de 540.000 registros, se reducirá a 20.000 registros, esto tiene como objetivo el no saturar la RAM.


In [ ]:
df = df.sample(n=20000, random_state=20)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 213839 to 504223
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Airline      20000 non-null  object
 1   AirportFrom  20000 non-null  object
 2   AirportTo    20000 non-null  object
 3   DayOfWeek    20000 non-null  int64 
 4   Time         20000 non-null  int64 
 5   Length       20000 non-null  int64 
 6   Delay        20000 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.2+ MB


#### Optimización de espacio

In [ ]:
# Función para verificar la memoria utilizada por el dataset en Mb.

def memoria_total(df):

    mem = df.memory_usage(deep=True).sum() / 1024**2
    return f"{mem:.2f} MB"

In [ ]:
# Función de "Downcasting", reduce tipo numérico "int64" al más pequeño posible.

def downcast_numericos(df):

    for col in df.select_dtypes(include=["int64"]).columns:
      df[col] = pd.to_numeric(df[col], downcast="integer")

    return df

In [ ]:
# Función de cambio de tipo de dato a "categoría" (definida por columnas).

def categorizar_columnas(df, columnas):

    for col in columnas:
        if col in df.columns:
            df[col] = df[col].astype("category")
        else:
            print(f"Columna '{col}' no encontrada en el DataFrame.")
    return df

In [ ]:
# Función para guardar en Parquet.

def guardar_parquet(df, nombre_archivo):

    df.to_parquet(f"{nombre_archivo}.parquet", engine="pyarrow", index=False)
    print(f"{nombre_archivo}.parquet guardado")

In [ ]:
# Función para limpiar caché (archivos basura).

import gc

def liberar_memoria():

    gc.collect()
    print("Memoria liberada")

In [ ]:
# Se verifica el "peso" de los datos antes y después del uso de las funciones anteriormente establecidas.

print(f"df → Uso antes: {memoria_total(df)}")

df = downcast_numericos(df)
df = categorizar_columnas(df, ["Airline", "AirportFrom", "AirportTo"])

print(f"df → Uso después: {memoria_total(df)}")

guardar_parquet(df, "df")
liberar_memoria()

df → Uso antes: 3.72 MB
df → Uso después: 0.41 MB
df.parquet guardado
Memoria liberada


### Ingeniería de datos

#### Creación de columnas hora y día de la semana

In [ ]:
# -----------------------------------------------------
# Crear columnas de hora y día de la semana
# -----------------------------------------------------

np.random.seed(42)
VELOCIDAD_PROMEDIO_KMH = 800

# -----------------------------------------------------
# Renombrar al español para un mejor entendimiento
# -----------------------------------------------------
df = df.rename(columns={
    'Airline': 'aerolinea',
    'AirportFrom': 'origen',
    'AirportTo': 'destino',
    'Length': 'duration_min',
    'Delay': 'retraso'  # solo para entrenamiento
})

# -----------------------------------------------------
# Calcular distancia en KM
# -----------------------------------------------------
df['distancia_km'] = (df['duration_min'] / 60) * VELOCIDAD_PROMEDIO_KMH

# -----------------------------------------------------
# Fechas base (DICIEMBRE 2018)
# -----------------------------------------------------
start_date = pd.to_datetime('2018-12-01')
end_date = pd.to_datetime('2018-12-31')
random_days = np.random.randint(0, (end_date - start_date).days + 1, size=len(df))
df['FlightDate'] = (start_date + pd.to_timedelta(random_days, unit='D')).normalize()

# -----------------------------------------------------
# FECHA/HORA de salida
# -----------------------------------------------------
df['DepartureDateTime'] = df['FlightDate'] + pd.to_timedelta(df['Time'], unit='m')

# -----------------------------------------------------
# Fecha partida formato ISO-8601 para modelo
# -----------------------------------------------------
df['fecha_partida'] = df['DepartureDateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# -----------------------------------------------------
# Eliminar columnas que ya no se usan
# -----------------------------------------------------
df = df.drop(columns=['duration_min'])

# -----------------------------------------------------
# Verificación de columnas
# -----------------------------------------------------
print("Columnas finales (incluye 'retraso' solo para entrenamiento):")
print(df.head())

Columnas finales (incluye 'retraso' solo para entrenamiento):
       aerolinea origen destino  DayOfWeek  Time  retraso  distancia_km  \
213839        OO    EGE     DEN          1   755        1    706.666667   
14809         MQ    DFW     CLE          3  1125        0   1933.333333   
221263        UA    DEN     SAN          1  1150        0   1960.000000   
194147        MQ    XNA     ORD          7   675        0   1400.000000   
234565        AA    BWI     MIA          2   930        0   2333.333333   

       FlightDate   DepartureDateTime        fecha_partida  
213839 2018-12-07 2018-12-07 12:35:00  2018-12-07T12:35:00  
14809  2018-12-20 2018-12-20 18:45:00  2018-12-20T18:45:00  
221263 2018-12-29 2018-12-29 19:10:00  2018-12-29T19:10:00  
194147 2018-12-15 2018-12-15 11:15:00  2018-12-15T11:15:00  
234565 2018-12-11 2018-12-11 15:30:00  2018-12-11T15:30:00  


In [ ]:
# -----------------------------------------------------
# Creación de columnas de hora y día de la semana
# -----------------------------------------------------
# El 'dia_semana' se conserva desde la columna original 'DayOfWeek'.
# La columna 'retraso' se mantiene solo para entrenamiento interno.

# Convertir 'fecha_partida' a datetime
df['fecha_partida_dt'] = pd.to_datetime(df['fecha_partida'])

# Crear columna de hora de salida como objeto time (HH:MM)
df['hora_salida'] = df['fecha_partida_dt'].dt.time

# Conservar día de la semana original desde 'DayOfWeek'
df['dia_semana'] = df['DayOfWeek'].astype('int8')  # del dataset original

# Reducir memoria: distancia y retraso
df['distancia_km'] = df['distancia_km'].astype('float32')
df['retraso'] = df['retraso'].astype('uint8')  # binario

# Eliminar columnas temporales redundantes
#if 'Time' in df.columns:                                                             NS: No eliminamos la columna Time, ya que mide la hora de salida en numeros enteros
#    df = df.drop(columns=['Time'])
df = df.drop(columns=['fecha_partida_dt'])

# Mantener solo columnas necesarias + 'retraso' para entrenamiento
df = df[['aerolinea', 'origen', 'destino', 'retraso', 'distancia_km', 'fecha_partida', 'dia_semana', 'hora_salida', 'Time']] #NS: Mantenemos Time para entrenar el modelo.

# Verificación rápida
print(df.head())
print(df.dtypes)

       aerolinea origen destino  retraso  distancia_km        fecha_partida  \
213839        OO    EGE     DEN        1    706.666687  2018-12-07T12:35:00   
14809         MQ    DFW     CLE        0   1933.333374  2018-12-20T18:45:00   
221263        UA    DEN     SAN        0   1960.000000  2018-12-29T19:10:00   
194147        MQ    XNA     ORD        0   1400.000000  2018-12-15T11:15:00   
234565        AA    BWI     MIA        0   2333.333252  2018-12-11T15:30:00   

        dia_semana hora_salida  Time  
213839           1    12:35:00   755  
14809            3    18:45:00  1125  
221263           1    19:10:00  1150  
194147           7    11:15:00   675  
234565           2    15:30:00   930  
aerolinea        category
origen           category
destino          category
retraso             uint8
distancia_km      float32
fecha_partida      object
dia_semana           int8
hora_salida        object
Time                int16
dtype: object


#### Creación columna fecha-hora

Se busca obtener datos de la API a partir de la fecha y hora del vuelo (ambos datos contenidos en una misma columna)

In [ ]:
# Se cambia el tipo de dato a "datetime" (fecha), de la columna "fecha_partida".

df["fecha_partida"] = pd.to_datetime(df["fecha_partida"])

In [ ]:
# Se crea una nueva columna con los datos de la columna "fecha_partida", redondeando la hora hacia abajo (ej: 12:40:00 = 12:00:00)
# con el fin obtener datos de la API apartir de la fecha y la hora.

df["fecha_hora_clima"] = df["fecha_partida"].dt.floor("h")

In [ ]:
# Se verifican los datos la columna creada.

df[["fecha_hora_clima"]].head()

,fecha_hora_clima
213839,2018-12-07 12:00:00
14809,2018-12-20 18:00:00
221263,2018-12-29 19:00:00
194147,2018-12-15 11:00:00
234565,2018-12-11 15:00:00


### Preparación para datos del clima



#### Creación diccionario "Latitud-Longitud"

Se crea un diccionario con el fin de extraer datos de la API con la ubicación de los aeropuertos.

In [ ]:
# Dataset con las ubicaciones de los aeropuertos del dataset a trabajar.

url2="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Aeropuertos.csv"
df_aeropuertos = pd.read_csv(url2)

df_aeropuertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84343 entries, 0 to 84342
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 84343 non-null  int64  
 1   ident              84343 non-null  object 
 2   type               84343 non-null  object 
 3   name               84343 non-null  object 
 4   latitude_deg       84343 non-null  float64
 5   longitude_deg      84343 non-null  float64
 6   elevation_ft       69745 non-null  float64
 7   continent          44928 non-null  object 
 8   iso_country        84051 non-null  object 
 9   iso_region         84343 non-null  object 
 10  municipality       79574 non-null  object 
 11  scheduled_service  84343 non-null  object 
 12  icao_code          9487 non-null   object 
 13  iata_code          9062 non-null   object 
 14  gps_code           43798 non-null  object 
 15  local_code         35849 non-null  object 
 16  home_link          441

In [ ]:
# Se conservan solo las columnas de latitud, longitud y codigo "Iata" que identifica al aeropuerto, con el fin de conectar
# nuestro dataset con los datos de la API de clima.

df_aeropuertos = df_aeropuertos[["latitude_deg", "longitude_deg", "iata_code"]]

In [ ]:
# Se verifica la creación de la tabla con las columnas anteriormente mencionadas.

df_aeropuertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84343 entries, 0 to 84342
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   latitude_deg   84343 non-null  float64
 1   longitude_deg  84343 non-null  float64
 2   iata_code      9062 non-null   object 
dtypes: float64(2), object(1)
memory usage: 1.9+ MB


In [ ]:
df.sample(5)

,aerolinea,origen,destino,retraso,distancia_km,fecha_partida,dia_semana,hora_salida,Time,fecha_hora_clima
351000,B6,BDL,MCO,0,2373.333252,2018-12-08 08:48:00,2,08:48:00,528,2018-12-08 08:00:00
468660,CO,ORD,IAH,1,2200.000000,2018-12-31 05:35:00,2,05:35:00,335,2018-12-31 05:00:00
10709,AA,PHL,DFW,1,3133.333252,2018-12-22 15:05:00,3,15:05:00,905,2018-12-22 15:00:00
306214,WN,BUF,MDW,1,1400.000000,2018-12-02 15:55:00,6,15:55:00,955,2018-12-02 15:00:00
34676,MQ,MIA,MEM,1,2066.666748,2018-12-27 20:40:00,4,20:40:00,1240,2018-12-27 20:00:00


In [ ]:
# Se obtiene el total de aeropuertos unicos, presentes en nuestro dataset de 20.000 registros.

airports_from = df["origen"].unique()
airports_to = df["destino"].unique()

todos_aeropuertos = set(airports_from) | set(airports_to)

len(todos_aeropuertos)

291

In [ ]:
# Se crea una tabla con las coordenadas de los aeropuertos de nuestro dataset de 20.000 registros.

df_aeropuertos_filtrado = df_aeropuertos[
    df_aeropuertos["iata_code"].isin(todos_aeropuertos)
    ][["iata_code", "latitude_deg", "longitude_deg"]]

df_aeropuertos_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 291 entries, 37905 to 64107
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   iata_code      291 non-null    object 
 1   latitude_deg   291 non-null    float64
 2   longitude_deg  291 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.1+ KB


In [ ]:
# Se toma un dato aleatorio para evaluación más adelante.

df_aeropuertos_filtrado.sample(n=1)

,iata_code,latitude_deg,longitude_deg
38237,CAE,33.938801,-81.119499


In [ ]:
#Se crea el diccionario, siendo su estructura "código iata: latitud, longitud"

dicc_coordenadas = {
    row["iata_code"]: {"lat": row["latitude_deg"], "lon": row["longitude_deg"]}
    for _, row in df_aeropuertos_filtrado.iterrows()
}

In [ ]:
# Se prueba el diccionario con un código presente en la tabla (dato aleatorio obtenido anteriormente).

print(dicc_coordenadas["CAE"])

{'lat': 41.504101, 'lon': -74.104797}


#### Registros únicos de aeropuertos

Esto tiene por objetivo el optimizar las llamadas a la API, evitando descargas innecesarias.

In [ ]:
 # Se obtienen las combinaciones únicas de aeropuerto y hora de partida, esto con el fin de reducir el número de consultas a la API de clima
# y evitar descargas innecesarias.

claves_clima = (
    df[["origen", "fecha_hora_clima"]]
    .dropna()
    .drop_duplicates()
)

In [ ]:
# Se crea una función que convierte los datos horarios de clima en un DataFrame de pandas.

def clima_json_a_df(data, aeropuerto):
    df = pd.DataFrame(data["hourly"])
    df["fecha_hora_clima"] = pd.to_datetime(df["time"]).dt.floor("h")
    df["origen"] = aeropuerto
    return df.drop(columns=["time"])

### Configuración y consumo de API

#### Configuración API de clima y función fallback

La API fue obtenida desde: https://open-meteo.com/

El objetivo de la función "Fallback", es el ser ejecutada como alternativa cuando la API falle.

In [ ]:
# Configuración de sesión HTTP con política de reintentos, función de llamada API de clima y función "Fallback".

api_historica = "https://archive-api.open-meteo.com/v1/archive"

# Política de reintentos para evitar fallos temporales de la API (que no caiga durante la obtención de datos)
sesion = requests.Session()
intentos = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[502, 503, 504]
)
sesion.mount("https://", HTTPAdapter(max_retries=intentos))

# Función Fallback, utilizada cuando la API falla en entregar datos.
def obtener_clima_fallback(fecha_salida_hora, fecha_llegada_hora):
    inicio = datetime.fromisoformat(fecha_salida_hora)
    fin = datetime.fromisoformat(fecha_llegada_hora)

    horas = int((fin - inicio).total_seconds() / 3600) + 1

    return {
        "hourly": {
            "time": [
                (inicio + timedelta(hours=i)).isoformat()
                for i in range(horas)
            ],
            "temperature_2m": [6.72] * horas,     #Valor obtenido a través del promedio de la columna con 540.000 registros (aprox)
            "windspeed_10m": [12.2] * horas,      #Valor obtenido a través del promedio de la columna con 540.000 registros (aprox)
            "weathercode": [7000],                #Valor obtenido a través del promedio de la columna con 540.000 registros (aprox)
        },
        "source": "fallback"
    }

# Función de llamada a la API, junto con función "fallback" en caso de falla.
def obtener_clima(lat, lon, fecha_salida_hora, fecha_llegada_hora):
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": fecha_salida_hora,
        "end_date": fecha_llegada_hora,
        "hourly": [
            "temperature_2m",
            "windspeed_10m",
            "weathercode"
        ],
        "timezone": "UTC"
    }

    try:
        respuesta = sesion.get(api_historica, params=params, timeout=30)
        respuesta.raise_for_status()

        return {
            "data": respuesta.json(),
            "is_fallback": False
        }

    except Exception as e:
        logging.warning(f"Fallo la API de clima, usando fallback | {e}")

        return {
            "data": obtener_clima_fallback(fecha_salida_hora, fecha_llegada_hora),
            "is_fallback": True
        }

#### LLamada a la API

In [ ]:
# A continuación, se hace uso de la función de la API de clima para su consumo.

dfs_clima = []

# Directorio de cache
cache_dir = "cache_clima"
os.makedirs(cache_dir, exist_ok=True)

# Fechas únicas por aeropuerto
claves_clima = df[["origen", "fecha_hora_clima"]].drop_duplicates()

for aeropuerto, grupo in claves_clima.groupby("origen"):

    if aeropuerto not in dicc_coordenadas:
        print(f"Aeropuerto {aeropuerto} no encontrado. Omitido.")
        continue

    coordenadas = dicc_coordenadas[aeropuerto]
    fechas = sorted(grupo["fecha_hora_clima"].dt.date.unique())

    i = 0
    while i < len(fechas):
        fecha_inicial = fechas[i]
        fecha_final = min(fecha_inicial + timedelta(days=6), fechas[-1])

        cache = f"{cache_dir}/clima_{aeropuerto}_{fecha_inicial}_{fecha_final}.pkl"

        if os.path.exists(cache):
            df_temporal = pd.read_pickle(cache)

        else:
            try:
                resultado = obtener_clima(
                    lat=coordenadas["lat"],
                    lon=coordenadas["lon"],
                    fecha_salida_hora=str(fecha_inicial),
                    fecha_llegada_hora=str(fecha_final)
                )

                # Mensaje "se usó fallback..."
                if resultado["is_fallback"]:
                    print(
                        f"Se usó fallback → "
                        f"{aeropuerto} | {fecha_inicial} a {fecha_final}"
                    )

                df_temporal = clima_json_a_df(
                    resultado["data"],
                    aeropuerto
                )

                df_temporal["fecha_hora_clima"] = (
                    pd.to_datetime(df_temporal["fecha_hora_clima"])
                      .dt.floor("h")
                )

                # Se guarda caché, siempre que no sea fallback
                if not resultado["is_fallback"]:
                    df_temporal.to_pickle(cache)

            except Exception as e:
                print(
                    f"Error descargando clima {aeropuerto} "
                    f"{fecha_inicial}-{fecha_final}: {e}"
                )
                i += 7
                continue

        dfs_clima.append(df_temporal)
        i += 7

# Concatenar todo
df_clima = pd.concat(dfs_clima, ignore_index=True)

/tmp/ipython-input-3610432459.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for aeropuerto, grupo in claves_clima.groupby("origen"):


### Integración de datos del clima

#### Unión de datos

In [ ]:
# Se unen las tablas

df_nuevo = df.merge(
    df_clima,
    on=["origen", "fecha_hora_clima"],
    how="left"
)

In [ ]:
# Se verifica que las tablas se hayan unido.

df_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   aerolinea         20000 non-null  category      
 1   origen            20000 non-null  object        
 2   destino           20000 non-null  category      
 3   retraso           20000 non-null  uint8         
 4   distancia_km      20000 non-null  float32       
 5   fecha_partida     20000 non-null  datetime64[ns]
 6   dia_semana        20000 non-null  int8          
 7   hora_salida       20000 non-null  object        
 8   Time              20000 non-null  int16         
 9   fecha_hora_clima  20000 non-null  datetime64[ns]
 10  temperature_2m    18903 non-null  float64       
 11  windspeed_10m     18903 non-null  float64       
 12  weathercode       18903 non-null  float64       
dtypes: category(2), datetime64[ns](2), float32(1), float64(3), int16(1), int8(1)

#### Limpieza y preparación de datos de la API

Durante la llamada a la API, se tiene el "código del clima", mientras que más adelante se renombra esta columna por "visibilidad" y se cambian los datos de esta columna por un aproximado en cuanto a la visibilidad en relación al código del clima.

La razón de esto es porque se hace uso de la API "histórica", en donde no se tienen datos relacionados a

In [ ]:
# Diccionario para estimar visibilidad según "weathercode" (código de clima).

visibilidad_estimacion = {
    0: 10000,   # Despejado
    1: 9000,    # Parcialmente nublado
    2: 8000,    # Nublado
    3: 6000,    # Lluvia ligera
    45: 7000,   # Neblina
    48: 5000,   # Neblina helada
    51: 6000,   # Llovizna ligera
    53: 5000,   # Llovizna moderada
    55: 4000,   # Llovizna intensa
    61: 5000,   # Lluvia ligera
    63: 4000,   # Lluvia moderada
    65: 3000,   # Lluvia intensa
    71: 4000,   # Nieve ligera
    73: 3000,   # Nieve moderada
    75: 2000,   # Nieve intensa
    80: 2500,   # Chubascos ligeros
    81: 1500,   # Chubascos moderados
    82: 1000,   # Chubascos intensos
    95: 500,    # Tormenta
    96: 400,    # Tormenta con granizo
    99: 300     # Tormenta intensa con granizo
}

In [ ]:
# Convierte los códigos de 'weathercode' a valores numéricos, los cuales tienen por objetivo representar la visibilidad.

df_nuevo["weathercode"] = df_nuevo["weathercode"].map(visibilidad_estimacion)

In [ ]:
# Se renombran las columnas (traducción al español, en el caso de "weathercode", esta representa la visibilidad).

df_nuevo = df_nuevo.rename(columns={
    'temperature_2m': 'temperatura',
    'windspeed_10m': 'velocidad_viento',
    "weathercode": "visibilidad"
})

In [ ]:
# Se verifica que los cambios anteriormente descritos se hayan efectuado

df_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   aerolinea         20000 non-null  category      
 1   origen            20000 non-null  object        
 2   destino           20000 non-null  category      
 3   retraso           20000 non-null  uint8         
 4   distancia_km      20000 non-null  float32       
 5   fecha_partida     20000 non-null  datetime64[ns]
 6   dia_semana        20000 non-null  int8          
 7   hora_salida       20000 non-null  object        
 8   Time              20000 non-null  int16         
 9   fecha_hora_clima  20000 non-null  datetime64[ns]
 10  temperatura       18903 non-null  float64       
 11  velocidad_viento  18903 non-null  float64       
 12  visibilidad       18903 non-null  float64       
dtypes: category(2), datetime64[ns](2), float32(1), float64(3), int16(1), int8(1)

In [ ]:
# Al observar la tabla anterior, se puede ver que faltan datos, al entregar una gran cantidad de datos a la API, esta entrega información por lotes
# lo que lleva a que en el proceso se pierdan algunos datos, para rellenar los datos faltantes, se obtiene el promedio de los datos obtenidos por
# columna y se rellenan los espacios vacios con dicho promedio.

promedio_temperatura = df_nuevo["temperatura"].mean()
promedio_velocidad_viento = df_nuevo["velocidad_viento"].mean()
promedio_visibilidad = round(df_nuevo["visibilidad"].mean(), -3)

df_nuevo["temperatura"] = df_nuevo["temperatura"].fillna(promedio_temperatura)
df_nuevo["velocidad_viento"] = df_nuevo["velocidad_viento"].fillna(promedio_velocidad_viento)
df_nuevo["visibilidad"] = df_nuevo["visibilidad"].fillna(promedio_visibilidad)

In [ ]:
# Se verifica que las columnas hayan sido rellenadas con los promedios anteriormente mencionados

df_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   aerolinea         20000 non-null  category      
 1   origen            20000 non-null  object        
 2   destino           20000 non-null  category      
 3   retraso           20000 non-null  uint8         
 4   distancia_km      20000 non-null  float32       
 5   fecha_partida     20000 non-null  datetime64[ns]
 6   dia_semana        20000 non-null  int8          
 7   hora_salida       20000 non-null  object        
 8   Time              20000 non-null  int16         
 9   fecha_hora_clima  20000 non-null  datetime64[ns]
 10  temperatura       20000 non-null  float64       
 11  velocidad_viento  20000 non-null  float64       
 12  visibilidad       20000 non-null  float64       
dtypes: category(2), datetime64[ns](2), float32(1), float64(3), int16(1), int8(1)

In [ ]:
# Se renombra el dataset nuevo a simplemente "df", esto es solo por motivos de comodidad.

df = df_nuevo

## PREPROCESAMIENTO

La variable "hora_salida" fue transformada a una representación numérica (hora_decimal) para permitir su uso en modelos basados en árboles, evitando problemas de conversión y manteniendo la información temporal relevante.

In [ ]:
# Convertir hora_salida a datetime
df['hora_salida'] = pd.to_datetime(
    df['hora_salida'],
    format='%H:%M:%S',
    errors='coerce'
)

# Crear hora decimal
df['hora_decimal'] = (
    df['hora_salida'].dt.hour +
    df['hora_salida'].dt.minute / 60
)

# Eliminar columna original
df.drop(columns=['hora_salida'], inplace=True)

### Definición de features finales

In [ ]:
# ============================
# PREP – Definición de features
# ============================

# Variables numéricas utilizadas por el modelo
numeric_features = [
    'distancia_km',
    'hora_decimal',
    'temperatura',
    'velocidad_viento',
    'visibilidad'
]

# Variables categóricas utilizadas por el modelo
categorical_features = [
    'aerolinea',
    'origen',
    'destino',
    'dia_semana'
]

# Variable objetivo
target = 'retraso'



#### Definición de variables de entrada

En esta etapa se definen explícitamente las variables utilizadas por el modelo final
de Machine Learning (Gradient Boosting).

Las variables numéricas representan información operativa del vuelo, la hora de salida
(expresada en formato decimal para facilitar su uso por el modelo) y condiciones
climáticas relevantes.  
Las variables categóricas capturan información propia del vuelo y su contexto temporal.

Este conjunto de variables corresponde al contrato de entrada del modelo en producción
y se mantiene consistente entre el entrenamiento y la inferencia.

In [ ]:
# ======================================
# PREPROCESAMIENTO - Manejo de valores faltantes
# ======================================

from sklearn.impute import SimpleImputer

# Imputador para variables numéricas
# Se utiliza la mediana por su robustez frente a valores atípicos
num_imputer = SimpleImputer(strategy="median")

# Ajuste del imputador sobre las variables numéricas
X_num = num_imputer.fit_transform(df[numeric_features])


#### Manejo de valores faltantes

Si bien el dataset utilizado para el entrenamiento del modelo ya cuenta con los valores
faltantes tratados, se documenta este paso como parte del pipeline de Machine Learning
para asegurar la robustez del modelo en producción.

En un entorno productivo, es posible que la información recibida desde la API no esté
completa. Por esta razón, el modelo contempla un proceso de imputación sobre las variables
numéricas, utilizando la mediana como estadístico de reemplazo por su estabilidad frente
a valores atípicos.

Este comportamiento se replica durante la inferencia para evitar errores en tiempo de ejecución.


### One-Hot Encoding de variables categóricas

In [ ]:
# ======================================
# PREPROCESAMIENTO - One-Hot Encoding
# ======================================

from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Encoder para variables categóricas
onehot_encoder = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

# Ajuste y transformación de variables categóricas
X_cat = onehot_encoder.fit_transform(df[categorical_features])

# Conversión a DataFrame para mantener nombres de columnas
X_cat = pd.DataFrame(
    X_cat,
    columns=onehot_encoder.get_feature_names_out(categorical_features),
    index=df.index
)


#### Codificación de variables categóricas

Las variables categóricas se transforman mediante One-Hot Encoding para convertirlas
en una representación numérica compatible con el modelo de Machine Learning.

El encoder se configura para ignorar categorías no vistas durante el entrenamiento,
lo que permite que el modelo sea robusto frente a nuevos valores recibidos en producción.

Este encoder es parte fundamental del pipeline y se serializa junto con el modelo,
ya que define la estructura exacta de las variables de entrada.

### Construcción del feature matrix final

In [ ]:
# ======================================
# PREPROCESAMIENTO - Feature Matrix final
# ======================================

import numpy as np

# Conversión de variables numéricas a DataFrame
X_num = pd.DataFrame(
    X_num,
    columns=numeric_features,
    index=df.index
)

# Concatenación de variables numéricas y categóricas
X = pd.concat([X_num, X_cat], axis=1)

# Variable objetivo
y = df[target]


#### Construcción de la matriz final de características

Una vez procesadas las variables numéricas y categóricas, ambas se concatenan para
construir la matriz final de características utilizada por el modelo.

Mantener el orden y la consistencia de las columnas es fundamental, ya que el modelo
espera recibir los datos en la misma estructura definida durante el entrenamiento.
Esta matriz representa la entrada directa al modelo en el entorno de producción.

## ENTRENAMIENTO

### Modelo final: Gradient Boosting

In [ ]:
# ======================================
# ENTRENAMIENTO - Modelo final
# ======================================

from sklearn.ensemble import GradientBoostingClassifier

# Definición del modelo final seleccionado
gb_model = GradientBoostingClassifier(
    random_state=42
)

# Entrenamiento del modelo sobre el dataset procesado
# gb_model.fit(X_train, y_train)


#### Entrenamiento del modelo Gradient Boosting

Durante la fase de experimentación se evaluaron distintos algoritmos de clasificación.
El modelo de Regresión Logística fue utilizado como base para la versión MVP.

Posteriormente, el modelo Gradient Boosting presentó un mejor desempeño general,
por lo que fue seleccionado como el modelo final (champion model) para producción.

En este notebook se documenta el entrenamiento del modelo final como referencia del
pipeline utilizado.


### Separación de datos (Train / Test)

In [ ]:
# ======================================
# ENTRENAMIENTO - Separación Train / Test
# ======================================

from sklearn.model_selection import train_test_split

# Separación de los datos en conjuntos de entrenamiento y prueba
# Se utiliza un split 80/20 para evaluar el desempeño del modelo
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


#### Separación de datos para entrenamiento y evaluación

El conjunto de datos se divide en subconjuntos de entrenamiento y prueba con el
objetivo de evaluar el desempeño del modelo sobre datos no vistos.

Se utiliza una división 80/20 y estratificación sobre la variable objetivo para
mantener la proporción de clases y evitar sesgos en la evaluación.


## EVALUACIÓN

In [ ]:
# ======================================
# EVALUACIÓN - Métricas del modelo
# ======================================

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report
)

# Predicciones del modelo sobre el conjunto de prueba
# y_pred = gb_model.predict(X_test)
# y_proba = gb_model.predict_proba(X_test)[:, 1]

# Cálculo de métricas
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# roc_auc = roc_auc_score(y_test, y_proba)

# Reporte de clasificación
# print(classification_report(y_test, y_pred))


#### Evaluación del modelo

El desempeño del modelo se evalúa utilizando un conjunto de prueba independiente,
empleando métricas estándar de clasificación.

Se consideran métricas como accuracy, precision, recall, F1-score y ROC-AUC, las
cuales permiten analizar tanto el rendimiento global del modelo como su capacidad
para identificar correctamente vuelos con retraso.

Estas métricas fueron utilizadas para comparar distintos modelos durante la fase
de experimentación y respaldar la selección del modelo final.


### Optimización del umbral de decisión

In [ ]:
# ======================================
# EVALUACIÓN - Optimización del umbral
# ======================================

import numpy as np
from sklearn.metrics import f1_score

# Evaluación de distintos umbrales sobre las probabilidades del modelo
# thresholds = np.arange(0.1, 0.9, 0.05)
# f1_scores = []

# for t in thresholds:
#     y_pred_threshold = (y_proba >= t).astype(int)
#     f1_scores.append(f1_score(y_test, y_pred_threshold))

# Selección del umbral que maximiza el F1-score
# best_threshold = thresholds[np.argmax(f1_scores)]


El modelo produce probabilidades de retraso, las cuales deben convertirse en una
decisión binaria mediante la definición de un umbral.

Durante la evaluación se analizaron distintos valores de umbral con el objetivo de
maximizar métricas relevantes para el negocio, como el F1-score, logrando un mejor
balance entre precisión y recall.

Este proceso no modifica el modelo entrenado, sino la forma en que se interpreta su
salida en producción.


### Calibración de probabilidades

In [ ]:
# ======================================
# EVALUACIÓN - Calibración de probabilidades
# ======================================

from sklearn.calibration import CalibratedClassifierCV

# Calibración de las probabilidades del modelo
# calibrated_model = CalibratedClassifierCV(
#     gb_model,
#     method="isotonic",
#     cv=5
# )

# calibrated_model.fit(X_train, y_train)


La calibración de probabilidades se aplica para asegurar que las probabilidades
predichas por el modelo reflejen de manera más fiel la frecuencia real de eventos.

Este ajuste es especialmente relevante en contextos donde las probabilidades son
consumidas por sistemas externos o utilizadas para la toma de decisiones.

La calibración se realizó como una capa adicional sobre el modelo entrenado, sin
alterar su estructura interna.


## IMPLEMENTACIÓN

### Serialización robusta del modelo (Bundle v2.0)

In [ ]:
# ======================================
# SERIALIZACIÓN ROBUSTA - Bundle v2.0
# ======================================

import joblib

# Guardado del modelo final (champion model)
joblib.dump(gb_model, "champion_model_v2.pkl")

# Guardado de objetos de preprocesamiento
joblib.dump(num_imputer, "num_imputer_v2.pkl")
joblib.dump(onehot_encoder, "onehot_encoder_v2.pkl")


['onehot_encoder_v2.pkl']

Como paso final del trabajo se serializan de manera
independiente los artefactos necesarios para la inferencia en producción.

El bundle incluye:
- El modelo Gradient Boosting final (champion model v2.0)
- El imputador de variables numéricas
- El encoder de variables categóricas

Esta separación permite una integración flexible con los componentes de Backend
y MLOps, garantizando reproducibilidad y control de versiones.

### Implementación del Microservicio con FastAPI

En esta sección se indica los pasos a realizar para la implementación del Microservicio FastAPI, en la cual se explica la estructura del proyecto y el contenido de los Scripts necesarios para implementar el poryecto, tanto en local, como en una VM (OCI, Render, AWS, etc.)

#### Estructura del proyecto

Esta es la estructura que debe tener el proyecto para poder replicar el funcionamiento, en caso de querer crear el proyecto de cero a partir de este Notebook, en otro caso, se puede acceder al [Repositorio del Proyecto](https://github.com/degartHub/nocountry-h12-25-equipo27-datascience) y clonarlo en su maquina, en la carpeta `flight-delay-api` ya se encuentra la estructura correcta.

```text
flight-delay-api/
├── app/
│   ├── __init__.py
│   ├── app.py
│   ├── inference_pipeline.py
│   ├── debug.py
│   ├── explainability/
│   │   ├── __init__.py
│   │   └── lime_service.py
│   └── weather/
│       ├── __init__.py
│       └── fallback.py

├── artifacts/
│   └── current/
│       ├── champion_model_v2.pkl
│       ├── lime_background_v2.pkl
│       ├── num_imputer_v2.pkl
│       ├── one_hot_encoder.pkl
│       └── metadata.json

├── tests/
│   ├── __init__.py
│   ├── test_api.py
│   ├── test_artifacts.py
│   └── test_inference.py

├── .github/
│   ├── cd.yml
│   └── ci.yml

├── .gitignore
├── docker-compose.yml
├── Dockerfile
├── requirements.txt
└── README.md
```




#### Dependencias del proyecto

El proyecto depende de las siguiente librerias (las cuales estarán también definidas en el `requierements.txt`)

```text
fastapi
pandas
numpy
scikit-learn
joblib
uvicorn
pydantic
pytest
httpx
prometheus-client
lime
```

#### Contenido de los archivos individuales

Todos los directorios creados para el proyecto con python, deben contener el modulo `__init__.py`, esto con el fin de que el entorno de ejecución python los reconozca como modulos Python, aún si estos modulos `__init__.py` estan vacios. Únicamente se debe garantizar su existencia.


---

Ahora iniciamos con el contenido de cada uno de los Scripts y archivos del proyecto.


##### `app.py`

---

```python
from fastapi import FastAPI, HTTPException, Response, Body
from pydantic import BaseModel, Field
from typing import Optional, List
import time
from prometheus_client import Counter, Histogram, generate_latest, CONTENT_TYPE_LATEST
from app.weather.fallback import apply_fallbacks
from app.inference_pipeline import predict, predict_batch, model
from app.debug import get_debug_info

# -----------------------
# APP
# -----------------------
app = FastAPI(
    title="Flight Delay Prediction API",
    version="2.0.5"
)

# -----------------------
# METRICAS PROMETHEUS
# -----------------------
REQUEST_COUNT = Counter(
    "api_requests_total",
    "Total number of API requests",
    ["endpoint"]
)

ERROR_COUNT = Counter(
    "api_errors_total",
    "Total number of API errors"
)

PREDICTION_LATENCY = Histogram(
    "prediction_latency_seconds",
    "Latency of prediction endpoint"
)

FALLBACK_COUNT = Counter(
    "fallback_total",
    "Total times any fallback was applied"
)

# -----------------------
# SCHEMAS PYDANTIC (SWAGGER)
# -----------------------
class PredictionInput(BaseModel):
    aerolinea: str
    origen: str
    destino: str
    fecha_partida: str

    distancia_km: float = Field(..., gt=0)
    temperatura: Optional[float] = Field(None, gt=-50, lt=60)
    velocidad_viento: Optional[float] = Field(None, ge=0)
    visibilidad: Optional[float] = Field(None, ge=0)


class PredictionOutput(BaseModel):
    prevision: str
    probabilidad: float
    latencia_ms: float
    explicabilidad: Optional[dict] = None


# -----------------------
# ENDPOINTS
# -----------------------
@app.post("/predict", response_model=List[PredictionOutput])
def predict_delay(
    data: List[PredictionInput] = Body(...),  # Siempre lista
    explain: bool = False
):
    REQUEST_COUNT.labels(endpoint="/predict").inc()
    start = time.perf_counter()

    try:
        payloads = [d.model_dump() for d in data]

        if len(payloads) == 1:
            # Single record → LIME permitido
            result = predict(payloads[0], explain=explain)
            latency_ms = (time.perf_counter() - start) * 1000
            result["latencia_ms"] = round(latency_ms, 2)
            return [result]  # Siempre devuelve lista para consistencia

        else:
            # Batch → no LIME
            results = predict_batch(payloads)
            latency_ms = (time.perf_counter() - start) * 1000
            for r in results:
                r["latencia_ms"] = round(latency_ms, 2)
            return results

    except Exception as e:
        ERROR_COUNT.inc()
        raise HTTPException(status_code=400, detail=str(e))


@app.get("/health")
def health_check():
    return {
        "status": "ok",
        "model_loaded": model is not None,
        "model_type": type(model).__name__ if model else None
    }


@app.get("/metrics")
def metrics() -> Response:
    return Response(
        generate_latest(),
        media_type=CONTENT_TYPE_LATEST
    )


@app.get("/", summary="Root endpoint para debugging y QA")
def root_debug():
    REQUEST_COUNT.labels(endpoint="/").inc()
    return get_debug_info()
```

##### `inference_pipeline.py`

---

```python
import joblib
import pandas as pd
from pathlib import Path
from typing import Dict, List
from app.weather.fallback import apply_fallbacks
from app.explainability.lime_service import get_top_3_influential_features

# -----------------------------
# RUTAS
# -----------------------------
BASE_DIR = Path(__file__).resolve().parent.parent
ARTIFACTS_DIR = BASE_DIR / "artifacts" / "current"

# -----------------------------
# CARGA DE ARTEFACTOS
# -----------------------------
model = joblib.load(ARTIFACTS_DIR / "champion_model_v2.pkl")
ohe = joblib.load(ARTIFACTS_DIR / "onehot_encoder_v2.pkl")
num_imputer = joblib.load(ARTIFACTS_DIR / "num_imputer_v2.pkl")

# -----------------------------
# DEFINICIÓN DE FEATURES
# -----------------------------
CATEGORICAL_FEATURES = [
    "aerolinea",
    "origen",
    "destino",
    "dia_semana"
]

NUMERIC_FEATURES = [
    "distancia_km",
    "hora_decimal",
    "temperatura",
    "velocidad_viento",
    "visibilidad"    
]

# -----------------------------
# PREPROCESAMIENTO BATCH
# -----------------------------
def preprocess_batch(payloads):
    if isinstance(payloads, list):
        df = pd.DataFrame(payloads)
    elif isinstance(payloads, pd.DataFrame):
        df = payloads.copy()
    else:
        raise ValueError("payloads debe ser lista de dicts o DataFrame")

    # Aplicar fallback fila por fila
    df = df.apply(lambda row: apply_fallbacks(row.to_dict()), axis=1, result_type='expand')
    df = df.drop(columns=["_fallback_used"], errors="ignore")  # CAMBIO: fix error 400

    # Fecha → hora_decimal y dia_semana
    dt = pd.to_datetime(df["fecha_partida"], errors="coerce")
    df["hora_decimal"] = dt.dt.hour + dt.dt.minute / 60
    df["dia_semana"] = dt.dt.dayofweek

    # Columnas faltantes
    for col in NUMERIC_FEATURES:
        if col not in df.columns:
            df[col] = 0.0
    for col in CATEGORICAL_FEATURES:
        if col not in df.columns:
            df[col] = "UNKNOWN"

    # Imputación numérica
    df[NUMERIC_FEATURES] = num_imputer.transform(df[NUMERIC_FEATURES])

    # OHE categórico
    X_cat = ohe.transform(df[CATEGORICAL_FEATURES])
    X_cat = pd.DataFrame(
        X_cat,
        columns=ohe.get_feature_names_out(CATEGORICAL_FEATURES),
        index=df.index
    )

    X_num = df[NUMERIC_FEATURES]
    X = pd.concat([X_num, X_cat], axis=1)
    return X

# -----------------------------
# PREDICCIÓN SINGLE RECORD
# -----------------------------
def predict(payload: Dict, explain: bool = False):
    X = preprocess_batch([payload])  # Reusar batch prep
    proba = model.predict_proba(X)[0, 1]

    threshold = 0.35
    prediction = "Retrasado" if proba >= threshold else "No Retrasado"

    result = {
        "prevision": prediction,
        "probabilidad": round(float(proba), 2)
    }

    if explain:
        lime_result = get_top_3_influential_features(X)
        result['explicabilidad'] = {
            'metodo': 'LIME',
            'top_3_features': lime_result['top_3_features_influyentes']
        }

    return result

# -----------------------------
# PREDICCIÓN BATCH
# -----------------------------
def predict_batch(payloads: List[Dict]):
    X = preprocess_batch(payloads)
    probas = model.predict_proba(X)[:, 1]

    threshold = 0.35
    predictions = ["Retrasado" if p >= threshold else "No Retrasado" for p in probas]

    results = []
    for i, p in enumerate(probas):
        result = {
            "prevision": predictions[i],
            "probabilidad": round(float(p), 2)
            # No LIME en batch
        }
        results.append(result)
    return results

```


##### `debug.py`

Este archivo corresponde al contenido del endpoint raíz `api/` esto con el fin de hacer debuggin humano y QA

---

```python
from app.inference_pipeline import model, CATEGORICAL_FEATURES, NUMERIC_FEATURES

def get_debug_info():
    return {
        "api_name": "Flight Delay Prediction API",
        "version": "2.0.2",
        "status": "ok",
        "model_loaded": model is not None,
        "model_type": type(model).__name__ if model else None,
        "features": {
            "categorical": CATEGORICAL_FEATURES,
            "numeric": NUMERIC_FEATURES
        },
        "prediction_threshold": 0.35,
        "example_payload": {
            "aerolinea": "AZ",
            "origen": "GIG",
            "destino": "GRU",
            "fecha_partida": "2025-11-10T14:30:00",
            "distancia_km": 350
        },
        "links": ["/predict", "/health", "/metrics"]
    }

```

##### `lime_service.py`

---

```python
mport pandas as pd
import numpy as np
import joblib
from pathlib import Path
from lime.lime_tabular import LimeTabularExplainer

# -----------------------------
# RUTAS
# -----------------------------
BASE_DIR = Path(__file__).resolve().parent.parent.parent
ARTIFACTS_DIR = BASE_DIR / "artifacts" / "current"

# -----------------------------
# CARGA ARTEFACTOS
# -----------------------------
model = joblib.load(ARTIFACTS_DIR / "champion_model_v2.pkl")

# Dataset de background (MISMO orden que X_train)
# ⚠️ Este archivo debe existir (subset del train ya preprocesado)
X_TRAIN_LIME = joblib.load(ARTIFACTS_DIR / "lime_background_v2.pkl")

FEATURE_NAMES = X_TRAIN_LIME.columns.tolist()

# -----------------------------
# LIME EXPLAINER (GLOBAL)
# -----------------------------
lime_explainer = LimeTabularExplainer(
    training_data=X_TRAIN_LIME.values,
    feature_names=FEATURE_NAMES,
    class_names=["No Retrasado", "Retrasado"],
    mode="classification",
    discretize_continuous=True
)

# -----------------------------
# FUNCIÓN PRINCIPAL (DA → PROD)
# -----------------------------
def get_top_3_influential_features(
    instance: pd.DataFrame,
    n_features: int = 12
) -> dict:
    """
    Implementación productiva EXACTA del protocolo DA.
    """

    instance_values = instance.values[0]

    explanation = lime_explainer.explain_instance(
        instance_values,
        model.predict_proba,
        num_features=n_features
    )

    pred_prob = model.predict_proba(instance)[0]
    pred_class = model.predict(instance)[0]

    prob_retraso = pred_prob[1]

    top_contributions = explanation.as_list()[:3]

    top_3 = []
    for feature_desc, weight in top_contributions:
        direction = (
            "a favor del retraso"
            if weight > 0
            else "en contra del retraso"
        )

        top_3.append({
            "feature": feature_desc,
            "weight": round(float(weight), 4),
            "direction": direction
        })

    return {
        "prediccion": int(pred_class),
        "prevision": "Retrasado" if pred_class == 1 else "No Retrasado",
        "probabilidad_retraso": round(float(prob_retraso), 4),
        "top_3_features_influyentes": top_3
    }

```

##### `fallback.py`

---

```python
from typing import Dict

# Todas las features que el modelo espera
DEFAULT_NUMERIC_FEATURES = {
    "distancia_km": 0.0,
    "temperatura": 0.0,
    "velocidad_viento": 5.0,
    "visibilidad": 10000.0,    
}

def apply_fallbacks(data: Dict) -> Dict:
    """
    Garantiza que todas las features numéricas requeridas
    por el modelo existan antes de crear el DataFrame.
    """
    enriched = data.copy()
    fallback_used = False

    for feature, default_value in DEFAULT_NUMERIC_FEATURES.items():
        if feature not in enriched or enriched[feature] is None:
            enriched[feature] = default_value
            fallback_used = True

    enriched["_fallback_used"] = fallback_used
    return enriched
```

##### `metadata.json`

A este archivo se le pueden hacer muchas mejoras para optimización de todo el servicio y una implementación de buenas prácticas.

---

```json
{
  "model_name": "flight_delay_gradient_boosting",
  "version": "2.0.0",
  "framework": "scikit-learn",
  "python_version": "3.11.9",
  "trained_at": "2026-01-12T14:22:00Z",
  "features": [
    "aerolinea",
    "origen",
    "destino",
    "distancia_km",
    "hora_decimal",
    "temperatura",
    "velocidad_viento",
    "visibilidad",
    "recent_delay_in_origin"
  ],
  "threshold": 0.35
}
```

##### `test_api.py`

---

```python
from fastapi.testclient import TestClient
from app.app import app

client = TestClient(app)

def test_predict_ok():
    payload = {
        "aerolinea": "AZ",
        "origen": "GIG",
        "destino": "GRU",
        "fecha_partida": "2025-11-10T14:30:00",
        "distancia_km": 350
    }

    response = client.post("/predict", json=payload)

    assert response.status_code == 200

    data = response.json()

    assert "prevision" in data
    assert "probabilidad" in data

    assert data["prevision"] in ["Retrasado", "No Retrasado"]
    assert 0.0 <= data["probabilidad"] <= 1.0
```

##### `test_artifacts.py`

---

```python
from app.inference_pipeline import model, ohe, scaler

def test_artifacts_loaded():
    """Testea que el modelo, scaler y encoder estan cargados correctamente."""
    assert model is not None, "Model is not loaded"
    assert ohe is not None, "One-Hot Encoder is not loaded"
    assert scaler is not None, "Scaler is not loaded"
```


##### `test_inference.py`

---

```python
from app.inference_pipeline import predict

payload = {
    "aerolinea": "AZ",
    "origen": "GIG",
    "destino": "GRU",
    "fecha_partida": "2025-11-10T14:30:00",
    "distancia_km": 350,
    "temperatura": 22,
    "velocidad_viento": 5,
    "visibilidad": 10000
}

result = predict(payload, explain=True)
print(result)

```

##### `.gitgnore`

Este archivo tiene la intención de evitar que los directorios y dependencias creadas por el entorno virtual de `Python` (`venv`, `Conda`, etc.) se suban al repositorio.



---



```
# Entornos virtuales
venv/
.env/
.env

# Byte-compiled
__pycache__/
*.pyc

# Pytest
.pytest_cache/

# VS Code
.vscode/

# OS
.DS_Store
Thumbs.db

# Logs
*.log

#keys
keys/
*.key
*.pub
```

##### `docker-compose.yml` LOCAL

Este archivo debe estar en la raíz del proyecto,para testear en docker local o en `Codespaces`

---

```yaml
services:
  api:
    build: .
    container_name: flight-delay-api
    ports:
      - "8000:8000"
    restart: unless-stopped


```

##### `docker-compose.yml` VM (OCI, AWS, Render, etc.)

Este archivo **NO** debe estar en el repositorio local, esta es la versión que se construye en la VM del servicio para `deploy` haciendo uso de *CI/CD* y `GitHub Actions`.

---

```yaml
version: "3.9"

services:
  api:
    image: ghcr.io/wolganstark/flight-delay-api:${IMAGE_TAG}
    container_name: flight-delay-api
    ports:
      - "8000:8000"
    restart: unless-stopped

```



##### `Dockerfile`

---

```
FROM python:3.11-slim

#Evita archivos .pyc y buffers


ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
WORKDIR /app

#Dependencias
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

#Codigo
COPY app/ ./app/
COPY artifacts/ ./artifacts/

EXPOSE 8000

CMD ["uvicorn", "app.app:app", "--host", "0.0.0.0", "--port", "8000"]
```

##### `requirements.txt`

---

```text
fastapi==0.123.10
pandas==2.2.2
numpy==2.0.2
scikit-learn==1.6.1
joblib==1.5.3
uvicorn==0.38.0
pydantic==2.12.3
pytest==9.0.2
httpx==0.28.1
prometheus-client==0.23.1
lime==0.2.0.1
```

### Mejoras futuras

El proyecto puede tener mejoras futuras que permitan una construcción más robusta o la implementación de más Features:



*   Prometheus: No la versión `prometheus-client`, si no la versión que genera las métricas del servicio, generalmente implementado en el puerto local 9090
*   Streamlit: Un dashboard con Streamlit para generar las métricas del negocio.
*   Metadata.json: Mejorar el `metadata.json` para la reproducibilidad del servicio y evitar inconvenientes con los errores del orden de las columnas y el uso de los `transformers`, esto incluye la refactorización del codigo haciendo uso de este archivo.

